In [1]:
import time

notebook_start_time = time.time()

## <span style="color:#ff5f27">👨🏻‍🏫 Create Deployment </span>

In this notebook, you'll create a deployment for your recommendation system.

**NOTE Currently the transformer scripts are not implemented.**

In [2]:
import sys
from pathlib import Path

root_dir = str(Path().absolute().parent)
if root_dir not in sys.path:
    sys.path.append(root_dir)

## <span style="color:#ff5f27">📝 Imports </span>

In [3]:
# TODO: How to adapt this with UV?
# !pip install -r requirements.txt

In [4]:
import warnings

warnings.filterwarnings("ignore")

from recsys import utils
from recsys.models.ranking_serving import RankingModelModule
from recsys.models.two_tower_serving import QueryModelModule

2024-11-09 20:30:32,621 WARNING: DeprecationWarning: ml_dtypes.float8_e4m3b11 is deprecated. Use ml_dtypes.float8_e4m3b11fnuz



## <span style="color:#ff5f27">🔮 Connect to Hopsworks Feature Store </span>

In [5]:
project, fs = utils.get_hopsworks_feature_store()

2024-11-09 20:30:33.650 | INFO     | recsys.utils:get_hopsworks_feature_store:10 - Loging to Hopsworks using HOPSWORKS_API_KEY env var.


Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/15551
Connected. Call `.close()` to terminate connection gracefully.


## <span style="color:#ff5f27">🚀 Ranking Model Deployment </span>


You start by deploying your ranking model. Since it is a CatBoost model you need to implement a `Predict` class that tells Hopsworks how to load the model and how to use it.

With that in place, you can finally deploy your model.

In [6]:
ranking_deployment = RankingModelModule.deploy_to_hopsworks(project)

Connected. Call `.close()` to terminate connection gracefully.


Uploading: 0.000%|          | 0/4306 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/1117 elapsed<00:00 remaining<?

Deployment created, explore it at https://c.app.hopsworks.ai:443/p/15551/deployments/324609
Before making predictions, start the deployment by using `.start()`


In [7]:
# Start the deployment
ranking_deployment.start()

  0%|          | 0/6 [00:00<?, ?it/s]

Start making predictions by using `.predict()`


In [8]:
# Check logs in case of failure
ranking_deployment.get_logs(component="transformer", tail=200)

Explore all the logs and filters in the Kibana logs at https://c.app.hopsworks.ai:443/p/15551/deployments/324609

Instance name: rankingdeployment-transformer-default-00001-deployment-58f7qnkc
2024-11-09 18:31:26.940 7 root INFO [<module>():180] Loading serving script
2024-11-09 18:31:29.949 7 root INFO [__init__():117] Initializing transformer for deployment: rankingdeployment
Connected. Call `.close()` to terminate connection gracefully.
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/srv/hops/hadoop-3.2.0.12-EE-RC0/share/hadoop/common/lib/log4j-slf4j-impl-2.19.0.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/srv/hops/hadoop-3.2.0.12-EE-RC0/share/hadoop/hdfs/lib/log4j-slf4j-impl-2.19.0.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
2024-11-09 18:31:39.127 

### <span style="color:#ff5f27">🚀 Test Ranking Model Deployment </span>


In [9]:
def get_top_recommendations(ranked_candidates, k=3):
    return [candidate[-1] for candidate in ranked_candidates["ranking"][:k]]

In [10]:
# Define a test input example
test_ranking_input = {
    "instances": [
        {
            "customer_id": "641e6f3ef3a2d537140aaa0a06055ae328a0dddf2c2c0dd6e60eb0563c7cbba0",
            "month_sin": 1.2246467991473532e-16,
            "query_emb": [
                0.214135289,
                0.571055949,
                0.330709577,
                -0.225899458,
                -0.308674961,
                -0.0115124583,
                0.0730511621,
                -0.495835781,
                0.625569344,
                -0.0438038409,
                0.263472944,
                -0.58485353,
                -0.307070434,
                0.0414443575,
                -0.321789205,
                0.966559,
            ],
            "month_cos": -1.0,
        }
    ]
}

# Test ranking deployment
ranked_candidates = ranking_deployment.predict(test_ranking_input)

# Retrieve article ids of the top recommended items
recommendations = get_top_recommendations(ranked_candidates, k=3)
recommendations

['744131003', '713108001', '710250001']

In [11]:
# Check logs in case of failure
ranking_deployment.get_logs(component="transformer", tail=200)

Explore all the logs and filters in the Kibana logs at https://c.app.hopsworks.ai:443/p/15551/deployments/324609

Instance name: rankingdeployment-transformer-default-00001-deployment-58f7qnkc
2024-11-09 18:31:47.412 7 opensearch INFO [log_request_success():235] POST https://rest.elastic.service.consul:9200/15551__embedding_default_project_embedding_0/_search [status:200 request:0.009s]
2024-11-09 18:31:47.422 7 opensearch INFO [log_request_success():235] POST https://rest.elastic.service.consul:9200/15551__embedding_default_project_embedding_0/_search [status:200 request:0.010s]
2024-11-09 18:31:47.429 7 opensearch INFO [log_request_success():235] POST https://rest.elastic.service.consul:9200/15551__embedding_default_project_embedding_0/_search [status:200 request:0.007s]
2024-11-09 18:31:47.440 7 opensearch INFO [log_request_success():235] POST https://rest.elastic.service.consul:9200/15551__embedding_default_project_embedding_0/_search [status:200 request:0.010s]
2024-11-09 18:31:47

## <span style="color:#ff5f27">🚀 Query Model Deployment </span>

Next, you'll deploy your query model.

In [12]:
query_model_deployment = QueryModelModule.deploy_to_hopsworks(project)

Connected. Call `.close()` to terminate connection gracefully.


Uploading: 0.000%|          | 0/2175 elapsed<00:00 remaining<?

Deployment created, explore it at https://c.app.hopsworks.ai:443/p/15551/deployments/324610
Before making predictions, start the deployment by using `.start()`


At this point, you have registered your deployment. To start it up you need to run:

In [13]:
# Start the deployment
query_model_deployment.start()

  0%|          | 0/6 [00:00<?, ?it/s]

Start making predictions by using `.predict()`


In [14]:
# Check logs in case of failure
query_model_deployment.get_logs(component="transformer", tail=20)

Explore all the logs and filters in the Kibana logs at https://c.app.hopsworks.ai:443/p/15551/deployments/324610

Instance name: querydeployment-transformer-default-00001-deployment-5f785d2j27
2024-11-09 18:32:33.973 7 root INFO [<module>():180] Loading serving script
2024-11-09 18:32:36.999 7 root INFO [__init__():117] Initializing transformer for deployment: querydeployment
Connected. Call `.close()` to terminate connection gracefully.
2024-11-09 18:32:37.963 7 root INFO [<module>():196] Starting KServe server
2024-11-09 18:32:37.964 7 root INFO [register_model():187] Registering model: querydeployment
2024-11-09 18:32:37.964 7 root INFO [start():129] Setting max asyncio worker threads as 12
2024-11-09 18:32:37.964 7 root INFO [serve():139] Starting uvicorn with 1 workers
2024-11-09 18:32:37.999 7 uvicorn.error INFO [serve():84] Started server process [7]
2024-11-09 18:32:37.999 7 uvicorn.error INFO [startup():45] Waiting for application startup.
2024-11-09 18:32:38.001 7 root INFO [

### <span style="color:#ff5f27">🚀 Test Query Model Deployment </span>

In [15]:
# Define a test input example
data = {
    "instances": {
        "customer_id": "641e6f3ef3a2d537140aaa0a06055ae328a0dddf2c2c0dd6e60eb0563c7cbba0",
        "transaction_date": "2022-11-15T12:16:25.330916",
    }
}

# Test the deployment
ranked_candidates = query_model_deployment.predict(data)

# Retrieve article ids of the top recommended items
recommendations = get_top_recommendations(ranked_candidates["predictions"], k=3)
recommendations

['916394001', '817477001', '595696030']

In [16]:
# Check logs in case of failure
query_model_deployment.get_logs(component="transformer",tail=200)

Explore all the logs and filters in the Kibana logs at https://c.app.hopsworks.ai:443/p/15551/deployments/324610

Instance name: querydeployment-transformer-default-00001-deployment-5f785d2j27
2024-11-09 18:32:33.973 7 root INFO [<module>():180] Loading serving script
2024-11-09 18:32:36.999 7 root INFO [__init__():117] Initializing transformer for deployment: querydeployment
Connected. Call `.close()` to terminate connection gracefully.
2024-11-09 18:32:37.963 7 root INFO [<module>():196] Starting KServe server
2024-11-09 18:32:37.964 7 root INFO [register_model():187] Registering model: querydeployment
2024-11-09 18:32:37.964 7 root INFO [start():129] Setting max asyncio worker threads as 12
2024-11-09 18:32:37.964 7 root INFO [serve():139] Starting uvicorn with 1 workers
2024-11-09 18:32:37.999 7 uvicorn.error INFO [serve():84] Started server process [7]
2024-11-09 18:32:37.999 7 uvicorn.error INFO [startup():45] Waiting for application startup.
2024-11-09 18:32:38.001 7 root INFO [

## <span style="color:#ff5f27"> Stop Deployments </span>

Stop the deployment when you're not using it.

In [17]:
# Stop the ranking model deployment
ranking_deployment.stop()

# Stop the query model deployment
query_model_deployment.stop()

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

---

In [18]:
# End the timer
notebook_end_time = time.time()

# Calculate and print the execution time
notebook_execution_time = notebook_end_time - notebook_start_time
print(f"⌛️ Notebook Execution time: {notebook_execution_time:.2f} seconds")

⌛️ Notebook Execution time: 158.88 seconds


---